In [1]:
import torch
import torch_pruning as tp
import torch.nn as nn
import torchvision.models as models

class AlexNetFineTuner(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetFineTuner, self).__init__()

        # Load pre-trained AlexNet
        self.model = models.alexnet(pretrained=False)
        self.model.classifier[6] = nn.Linear(4096, num_classes)  # Update the classifier layer

    def prune_model(self, pruning_percentage=0.2):
        # Prepare example inputs for building the dependency graph
        example_inputs = torch.randn(1, 3, 224, 224, dtype=torch.float32)

        print("MODEL BEFORE PRUNING:\n", self.model)

        # Build the Dependency Graph
        DG = tp.DependencyGraph().build_dependency(self.model, example_inputs)

        # Layers to prune
        layers_to_prune = {
            "conv2": self.model.features[3],    # second conv layer in AlexNet
            "conv3": self.model.features[6],
            "conv4": self.model.features[8],
            "conv5": self.model.features[10],
            "fc1": self.model.classifier[1],
            "fc2": self.model.classifier[4]
        }

        # Helper function to compute pruning indices based on magnitude
        def get_pruning_indices(module, percentage):
            with torch.no_grad():
                weight = module.weight.data
                if isinstance(module, nn.Conv2d):
                    channel_norms = weight.abs().mean(dim=[1, 2, 3])  # Compute L1 norm of each output channel
                elif isinstance(module, nn.Linear):
                    channel_norms = weight.abs().mean(dim=1)  # Compute L1 norm of each output feature
                else:
                    return None

                pruning_count = int(channel_norms.size(0) * percentage)
                if pruning_count == 0:
                    return []  # No pruning if percentage is too small

                _, prune_indices = torch.topk(channel_norms, pruning_count, largest=False)
                return prune_indices.tolist()

        # Collect pruning groups for all layers
        groups = []
        for layer_name, layer_module in layers_to_prune.items():
            if isinstance(layer_module, nn.Conv2d):
                prune_fn = tp.prune_conv_out_channels
            elif isinstance(layer_module, nn.Linear):
                prune_fn = tp.prune_linear_out_channels
            else:
                print(f"Skipping {layer_name}: Unsupported layer type {type(layer_module)}")
                continue

            pruning_idxs = get_pruning_indices(layer_module, pruning_percentage)
            if pruning_idxs is None or len(pruning_idxs) == 0:
                print(f"No channels to prune for {layer_name}.")
                continue

            group = DG.get_pruning_group(layer_module, prune_fn, idxs=pruning_idxs)
            if DG.check_pruning_group(group):
                groups.append((layer_name, group))
            else:
                print(f"Invalid pruning group for layer {layer_name}, skipping pruning.")

        # Prune all layers together if we have at least one valid group
        if groups:
            print(f"Pruning with {pruning_percentage*100}% percentage on {len(groups)} layers...")
            for layer_name, group in groups:
                print(f"Pruning layer: {layer_name}")
                group.prune()

            print("MODEL AFTER PRUNING:\n", self.model)
        else:
            print("No valid pruning groups found. The model was not pruned.")

# Example usage
def main():
    model = AlexNetFineTuner()
    model.prune_model(pruning_percentage=0.2)

if __name__ == "__main__":
    main()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MODEL BEFORE PRUNING:
 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_feat